In [1]:
import requests
import sys
import json

def process_image(image_path, server_url='http://localhost:5000/process-image'):
    """
    Send an image file to the image processing server and print the results.
    
    :param image_path: Path to the image file to be processed
    :param server_url: URL of the image processing server (default: http://localhost:5000/process-image)
    """
    try:
        with open(image_path, 'rb') as image_file:
            files = {'image': (image_path, image_file)}
            response = requests.post(server_url, files=files)

        if response.status_code == 200:
            result = response.json()
            print("Extracted Text:")
            print(result['extracted_text'])
            print("\nClaude's Analysis:")
            print(result['claude_analysis'])
        else:
            print(f"Error: Server returned status code {response.status_code}")
            print(response.text)
    
    except FileNotFoundError:
        print(f"Error: File not found - {image_path}")
    except requests.RequestException as e:
        print(f"Error: Failed to send request - {e}")
    except json.JSONDecodeError:
        print("Error: Failed to parse server response")
    except KeyError:
        print("Error: Unexpected response format from server")

if __name__ == "__main__":
    if len(sys.argv) < 2:
        print("Usage: python client.py <path_to_image_file>")
        sys.exit(1)

    image_path = sys.argv[1]
    process_image(image_path)

Error: File not found - --f=/home/blagden/.local/share/jupyter/runtime/kernel-v2-12241tX63PXWzdYiO.json
